In [1]:
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

In [2]:
transform = transforms.Compose([
    transforms.Resize((32, 32)),          # Resize to match model input size
    transforms.RandomHorizontalFlip(),    # Flip horizontally for variety
    transforms.RandomRotation(15),        # Small rotations
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Adjust colors
    transforms.ToTensor(),                # Convert to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize
])

dataset_path = './Dataset'
dataset = datasets.ImageFolder(root=dataset_path, transform=transform)

In [3]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_data, test_data = random_split(dataset, [train_size, val_size])

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True, num_workers=2)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=32, shuffle=False, num_workers=2)

In [4]:
image, label = train_data[0]

In [5]:
image.size()

torch.Size([3, 32, 32])

In [6]:
class_names = dataset.classes

In [7]:
class NeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.conv1 = nn.Conv2d(3, 15, 5)
        self.bn1 = nn.BatchNorm2d(15)
        self.conv2 = nn.Conv2d(15, 30, 5)
        self.bn2 = nn.BatchNorm2d(30)
        self.conv3 = nn.Conv2d(30, 60, 3)  # New convolutional layer
        self.bn3 = nn.BatchNorm2d(60)
        
        self.pool = nn.MaxPool2d(2, 2)
        
        self.fc1 = nn.Linear(60, 120)  # Adjusted for the new layer
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 20)
        
        self.dropout = nn.Dropout(0.5)  # Dropout to prevent overfitting

    def forward(self, x):
        x = self.pool(F.relu(self.bn1(self.conv1(x))))
        x = self.pool(F.relu(self.bn2(self.conv2(x))))
        x = self.pool(F.relu(self.bn3(self.conv3(x))))  # Apply new layer
        x = torch.flatten(x, 1)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [8]:
net = NeuralNet()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.001)

In [9]:
for epoch in range(10):
    print(f'Training epoch {epoch}...')

    running_loss = 0.0
    
    for i, data in enumerate(train_loader):
        inputs, labels = data
        
        optimizer.zero_grad()
        
        outputs = net(inputs)
        
        loss = loss_function(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()

    print(f'Loss: {running_loss / len(train_loader):.4f}')

Training epoch 0...
Loss: 2.9889
Training epoch 1...
Loss: 2.9211
Training epoch 2...
Loss: 2.8237
Training epoch 3...
Loss: 2.7830
Training epoch 4...
Loss: 2.7094
Training epoch 5...
Loss: 2.6636
Training epoch 6...
Loss: 2.5852
Training epoch 7...
Loss: 2.5794
Training epoch 8...
Loss: 2.5587
Training epoch 9...
Loss: 2.4926


In [10]:
torch.save(net.state_dict(), 'trained_net.pth')

In [11]:
net = NeuralNet()
net.load_state_dict(torch.load('trained_net.pth'))

C:\Users\Bhargava K K\AppData\Local\Temp\ipykernel_11444\1690949604.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('trained_net.pth'))


<All keys matched successfully>

In [12]:
correct = 0
total = 0

net.eval()

with torch.no_grad():
    for data in test_loader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total

print(f'Accuracy: {accuracy}%')

Accuracy: 14.166666666666666%


In [13]:
new_transforms = transforms.Compose([
    transforms.Resize((32, 32)),          # Resize to match model input size
    transforms.RandomHorizontalFlip(),    # Flip horizontally for variety
    transforms.RandomRotation(15),        # Small rotations
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2),  # Adjust colors
    transforms.ToTensor(),                # Convert to Tensor
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))  # Normalize  
])

In [14]:
def load_image(image_path):
    image = Image.open(image_path)
    image = new_transforms(image)
    image = image.unsqueeze(0)
    return image

image_paths = ['wolf.jpg', 'hyena.jpg']
images = [load_image(img) for img in image_paths]

net.eval()
with torch.no_grad():
    for image  in images:
        output = net(image)
        _, predicted = torch.max(output, 1)
        print(f'Prediction: {class_names[predicted.item()]}')

Prediction: chimpanzee
Prediction: lion
